In [4]:
import pandas as pd
import os
from tqdm.auto import tqdm

In [5]:
# ==============================================================================
# ФУНКЦИИ ЗАГРУЗКИ ВНЕШНИХ ДАННЫХ
# ==============================================================================

def load_central_bank_rates(filepath: str) -> pd.DataFrame:
    """
    Загружает и подготавливает данные по ключевой ставке ЦБ РФ из Excel-файла.
    """
    print(f"Загрузка данных по ставке ЦБ из: {filepath}")
    try:
        # Используем dayfirst=True, чтобы правильно распознать формат ДД.ММ.ГГГГ
        rates_df = pd.read_excel(filepath, parse_dates=['date'])
        
        # Переименовываем колонки для единообразия
        rates_df.rename(columns={'date': 'Date', 'rate': 'cbr_rate'}, inplace=True)
        
        # Убираем время, оставляем только дату
        rates_df['Date'] = rates_df['Date'].dt.date
        rates_df['Date'] = pd.to_datetime(rates_df['Date'])
        
        # Удаляем дубликаты дат, если они есть
        rates_df.drop_duplicates(subset='Date', inplace=True)
        
        # Заменяем запятые на точки и преобразуем в числовой формат
        if rates_df['cbr_rate'].dtype == 'object':
            rates_df['cbr_rate'] = rates_df['cbr_rate'].str.replace(',', '.').astype(float)
        
        print("Данные по ставке успешно загружены и обработаны.")
        return rates_df
        
    except FileNotFoundError:
        print(f"ОШИБКА: Файл со ставками не найден по пути {filepath}.")
        return None
    except Exception as e:
        print(f"ОШИБКА при обработке файла со ставками: {e}")
        return None



In [6]:

# ==============================================================================
# ОСНОВНОЙ СКРИПТ
# ==============================================================================

# --- НАСТРОЙКИ ---
data_folder = "../data/"
raw_data_filename = 'moex_raw_data.csv'
rates_filename = 'ru_central_bank_rates.xlsx'
output_filename = 'moex_data_with_external.csv'

rate_folder = "../external_data/"

# --- ШАГ 1: Загрузка сырых данных по акциям ---
print(f"Загрузка сырых данных по акциям из: {data_folder + raw_data_filename}")
try:
    stocks_df = pd.read_csv(os.path.join(data_folder, raw_data_filename), parse_dates=['Date'])
    print(f"Загружено {len(stocks_df)} строк данных по акциям.")
except FileNotFoundError:
    print("ОШИБКА: Файл с сырыми данными не найден. Запустите сначала download_data.py.")
    raise

# --- ШАГ 2: Загрузка данных по ставке ЦБ ---
rates_df = load_central_bank_rates(os.path.join(rate_folder, rates_filename))

if rates_df is None:
    raise ValueError("No rates")

# --- ШАГ 3: Обогащение данных (Объединение таблиц) ---
print("\nОбогащение данных: добавление ставки ЦБ к каждой дате...")

# Используем метод merge_asof. Он идеально подходит для таких задач.
# Он находит для каждой даты в левой таблице (акции)
# последнюю известную дату в правой таблице (ставки).

# Сортируем обе таблицы по дате - это обязательное требование для merge_asof
stocks_df.sort_values(by='Date', inplace=True)
rates_df.sort_values(by='Date', inplace=True)

enriched_df = pd.merge_asof(
    left=stocks_df,
    right=rates_df,
    on='Date'
)
enriched_df['cbr_rate'].fillna(-1, inplace=True)



print("Данные успешно обогащены.")

# --- ШАГ 4: Сохранение результата ---
print("\n--- Итоговый обогащенный DataFrame ---")
print(enriched_df.info())


print("\nПервые 5 строк с добавленной ставкой:")
print(enriched_df.head())

# Посмотрим, как выглядит результат
print("\nПоследние 5 строк с добавленной ставкой:")
print(enriched_df.tail())

print(f"\nСохранение обогащенных данных в файл: {data_folder + output_filename}")
enriched_df.to_csv(os.path.join(data_folder, output_filename), index=False)
print("Обогащенные данные успешно сохранены!")

Загрузка сырых данных по акциям из: ../data/moex_raw_data.csv
Загружено 187827 строк данных по акциям.
Загрузка данных по ставке ЦБ из: ../external_data/ru_central_bank_rates.xlsx
Данные по ставке успешно загружены и обработаны.

Обогащение данных: добавление ставки ЦБ к каждой дате...
Данные успешно обогащены.

--- Итоговый обогащенный DataFrame ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187827 entries, 0 to 187826
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   Date      187827 non-null  datetime64[ns]
 1   Ticker    187827 non-null  object        
 2   Open      185113 non-null  float64       
 3   High      185113 non-null  float64       
 4   Low       185113 non-null  float64       
 5   Close     185113 non-null  float64       
 6   Volume    187827 non-null  int64         
 7   cbr_rate  187827 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory u

/var/folders/t9/st88xs4n5dv45285_22l27v9977913/T/ipykernel_18680/3719055614.py:12: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  rates_df = pd.read_excel(filepath, parse_dates=['date'])
/var/folders/t9/st88xs4n5dv45285_22l27v9977913/T/ipykernel_18680/1889653885.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  enriched_df['cbr_rate'].fillna(-1, inplace=True)


Обогащенные данные успешно сохранены!
